In [1]:
#!/usr/bin/env python3
import os, sys, signal,rospy, argparse, csv

from tqdm import tqdm
import numpy as np
from scipy.spatial.transform import Rotation as R
from scipy.spatial.distance import cdist
import copy, pickle
import open3d as o3d
import open3d.visualization as vis
import torch
from torch.autograd import Variable

from open3d_ros_helper import open3d_ros_helper as orh
from geometry_msgs.msg import Pose, PoseArray, Point # PoseArray, Pose

from models.mdgat import MDGAT

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
parser = argparse.ArgumentParser(
    description='Point cloud matching and pose evaluation',
    formatter_class=argparse.ArgumentDefaultsHelpFormatter)

opt = argparse.Namespace(
    slam_dir = '/media/vision/Seagate/DataSets/KRGM/kitti/',
    data_folder = 'harris_3D',
    local_global = False,
    seq_num = '00',
    visualize = False,
    vis_line_width = 0.2,
    calculate_pose = True,
    learning_rate = 0.0001,
    batch_size = 1,
    train_path = './KITTI/',
    model_out_path = './models/checkpoint',
    memory_is_enough = True,
    local_rank = 0,
    txt_path = './KITTI/preprocess-random-full',
    keypoints_path = './KITTI/keypoints/tsf_256_FPFH_16384-512-k1k16-2d-nonoise',
    resume_model = './checkpoint/kitti/mdgat-l9-gap_loss-pointnetmsg-04_01_19_32/train_step3/nomutualcheck-mdgat-batch16-gap_loss-pointnetmsg-USIP-04_01_19_32/best_model_epoch_221(val_loss0.31414552026539594).pth',
    loss_method = 'triplet_loss',
    net = 'mdgat',
    mutual_check = False,
    k = [128, None, 128, None, 64, None, 64, None],
    l = 9,
    descriptor = 'pointnetmsg',
    keypoints = 'USIP',
    ensure_kpts_num = False,
    max_keypoints = -1,
    match_threshold = 0.2,
    threshold = 0.5,
    triplet_loss_gamma = 0.5,
    sinkhorn_iterations = 20,
    train_step = 3,

    fpfh_normal_radiuse = 0.3,
    fpfh_descriptors_radiuse = 1.0,
    seq_list = [0],
    mdgat_path = './KITTI',
    kitti_path = '/media/vision/Seagate/DataSets/kitti/dataset/sequences',
    transform_opt = 0
)

In [3]:
class dataset():
    def __init__(self, args) -> None:
        self.gt_seq = args.seq_num
        self.data_folder = args.data_folder

        self.dir_SLAM_path = args.slam_dir + self.data_folder +"/"
        self.descriptor_type = args.descriptor

        self.poses = []
        self.dense_scans = []
        self.keypoints = []
        self.descriptors = []
        self.local_graph_range = [0, 0]
        self.divided_keypoints = np.array([])
        self.divided_dense_scans = []
        self.divided_seq_num = 0

        self._get_SLAM_poses()
        self._get_dense_frames()
        self._get_keypoints()
        self._get_descriptors()
        print("[Load] %d's poses SLAM data loaded" % len(self.poses))

    def _get_SLAM_poses(self):
        with open(file=os.path.join(self.dir_SLAM_path, "Poses_kitti_" + self.gt_seq + ".pickle"), mode='rb') as f:
            self.poses = pickle.load(f)

    def _get_dense_frames(self):
        with open(file=os.path.join(self.dir_SLAM_path, "DenseFrames_kitti_" + self.gt_seq + ".pickle"), mode='rb') as f:
            self.dense_scans = pickle.load(f)

    def _get_keypoints(self):
        with open(file=os.path.join(self.dir_SLAM_path, "keyPoints_kitti_" + self.gt_seq + ".pickle"), mode='rb') as f:
            self.keypoints = pickle.load(f) # (pose_num, keypoint_num, 3), (keypoint_num, 3)는 np.array

    def _get_descriptors(self):
        if self.descriptor_type == "FPFH":
            with open(file=os.path.join(self.dir_SLAM_path, "Descriptors_FPFH_kitti_" + self.gt_seq + ".pickle"), mode='rb') as f:
                self.descriptors = pickle.load(f)
        elif self.descriptor_type == "pointnet" or "pointnetmsg":
            for kps in self.keypoints:
                self.descriptors.append(torch.zeros((kps.shape[0], 128)))

    def divide_data(self, divide_num = 256):
        # divide keypoints
        flatten_keypoints = []
        for kps in self.keypoints:
            for kp in kps:
                flatten_keypoints.append(np.array(kp))
        flatten_keypoints = np.array(flatten_keypoints) # (n, 3)
        sample_num = flatten_keypoints.shape[0] // (divide_num)
        flatten_keypoints = flatten_keypoints[:(divide_num) * sample_num]
        self.divided_keypoints = flatten_keypoints.reshape((sample_num, divide_num, 3))

        # devide dense_scans
        n = 0
        divided_pcd = o3d.geometry.PointCloud()
        for idx, kps in enumerate(self.keypoints):
            divided_pcd.points.extend(self.dense_scans[idx])
            for _ in kps:
                n += 1
                if n % divide_num == 0:
                    divided_pcd = divided_pcd.voxel_down_sample(voxel_size=0.2)
                    self.divided_dense_scans.append(np.array(divided_pcd.points))
                    divided_pcd.clear()
                    divided_pcd.points.extend(self.dense_scans[idx])
                    n = 0

        self.divided_seq_num = sample_num
        self.descriptors = [None] * sample_num
        print("[Divide] Keypoints and dense scan divided into %d-num seq" % (sample_num))

    def get_divided_data(self, idx):
        if idx >= len(self.divided_keypoints) - 1:
            idx = len(self.divided_keypoints) - 1
            print("[Warning] idx is over the divided keypoints, so return the last one")
        infer_kp = torch.tensor(self.divided_keypoints[idx], dtype=torch.double).unsqueeze(0)
        infer_pc = torch.tensor(self.divided_dense_scans[idx], dtype=torch.double)
        infer_pc = torch.cat((infer_pc, torch.ones(infer_pc.shape[0], 1)), dim=1)
        
        n = infer_pc.shape[0] // 2
        infer_pc = infer_pc[:n*2,:]
        infer_pc = infer_pc.reshape((-1, 8)).unsqueeze(0)
        
        infer_scores = torch.ones((infer_kp.shape[1])).unsqueeze(0)
        
        return{
            'keypoints0': infer_kp,
            'cloud0': infer_pc,
            'scores0': infer_scores
        } 
    
    def push_descriptor(self, idx, descriptors):
        if idx >= len(self.descriptors):
            idx = len(self.descriptors) - 1
            print("[Warning] idx is over the divided keypoints, so return the last one")
        
        self.descriptors[idx] = descriptors.cpu()

    def get_matching_data(self, idx):
        if idx >= len(self.divided_keypoints) - 1:
            idx = len(self.divided_keypoints) - 1
            print("[Warning] idx is over the divided keypoints, so return the last one")
        if idx < 2:
            idx = 2
            print("[Warning] idx's minimum value is 2, so return same data with input idx 2")

        local_keypoints = self.divided_keypoints[idx]
        local_descriptors = self.descriptors[idx]
        local_pc = self.divided_dense_scans[idx]

        for i in [idx + 1]:
            local_keypoints = np.vstack((local_keypoints, self.divided_keypoints[i]))
            local_descriptors = torch.cat((local_descriptors, self.descriptors[i]), dim=2)
            local_pc = np.vstack((local_pc, self.divided_dense_scans[i]))

        global_keypoints = self.divided_keypoints[0]
        global_descriptors = self.descriptors[0]
        global_pc = self.divided_dense_scans[0]

        # for i in range(1, 2):
        for i in [1]:
            global_keypoints = np.vstack((global_keypoints, self.divided_keypoints[i]))
            global_descriptors = torch.cat((global_descriptors, self.descriptors[i]), dim=2)
            global_pc = np.vstack((global_pc, self.divided_dense_scans[i]))

        local_pc = torch.tensor(local_pc, dtype=torch.double)
        local_pc = torch.cat((local_pc, torch.ones(local_pc.shape[0], 1)), dim=1)
        global_pc = torch.tensor(global_pc, dtype=torch.double)
        global_pc = torch.cat((global_pc, torch.ones(global_pc.shape[0], 1)), dim=1)

        dists = cdist(global_keypoints, local_keypoints)
        min1 = np.argmin(dists, axis=0)
        min2 = np.argmin(dists, axis=1)
        min1v = np.min(dists, axis=1)
        min1f = min2[min1v < 0.5]
        rep = len(min1f)
        match1, match2 = -1 * np.ones((len(global_keypoints)), dtype=np.int16), -1 * np.ones((len(local_keypoints)), dtype=np.int16)
        match1[min1v < 0.5] = min1f
        min2v = np.min(dists, axis=0)
        min2f = min1[min2v < 0.5]
        match2[min2v < 0.5] = min2f

        global_keypoints = torch.tensor(global_keypoints, dtype=torch.double).unsqueeze(0)
        local_keypoints = torch.tensor(local_keypoints, dtype=torch.double).unsqueeze(0)

        return {
            'keypoints0': global_keypoints,
            'keypoints1': local_keypoints,
            'descriptors0': global_descriptors,
            'descriptors1': local_descriptors,
            'cloud0': global_pc,
            'cloud1': local_pc,
            'gt_matches0': match1,
            'gt_matches1': match2,
            'rep': rep
        }
                                     
    def __len__(self):
        return len(self.poses)

emplimentation 테스트 시나리오

In [4]:
# 1. 데이터 로드 및 분할
dataset = dataset(opt)
dataset.divide_data(256)

[Load] 929's poses SLAM data loaded
[Divide] Keypoints and dense scan divided into 42-num seq


In [5]:
pred = dataset.get_divided_data(2)
print(pred['keypoints0'].shape, pred['cloud0'].shape, pred['scores0'].shape)    

torch.Size([1, 256, 3]) torch.Size([1, 54621, 8]) torch.Size([1, 256])


In [6]:
# 2. 모델 로드
path_checkpoint = opt.resume_model  
checkpoint = torch.load(path_checkpoint, map_location={'cuda:2':'cuda:0'})  
lr = checkpoint['lr_schedule']
config = {
    'net': {
        'sinkhorn_iterations': opt.sinkhorn_iterations,
        'match_threshold': opt.match_threshold,
        'lr': opt.learning_rate,
        'loss_method': opt.loss_method,
        'k': opt.k,
        'descriptor': opt.descriptor,
        'mutual_check': opt.mutual_check,
        'triplet_loss_gamma': opt.triplet_loss_gamma,
        'train_step':opt.train_step,
        'L':opt.l
    }
}
net = MDGAT(config.get('net', {}))
optimizer = torch.optim.Adam(net.parameters(), lr=config.get('net', {}).get('lr'))
net = torch.nn.DataParallel(net)
net.load_state_dict(checkpoint['net'])

if torch.cuda.is_available():
    # torch.cuda.set_device(opt.local_rank)
    device=torch.device('cuda:{}'.format(opt.local_rank))
else:
    device = torch.device("cpu")
    print("### CUDA not available ###")

net.to(device)

DataParallel(
  (module): MDGAT(
    (penc): PointnetEncoderMsg(
      (sa1): PointNetSetKptsMsg(
        (conv_blocks): ModuleList(
          (0): ModuleList(
            (0): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
            (1): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1))
            (2): Conv2d(32, 64, kernel_size=(1, 1), stride=(1, 1))
          )
          (1): ModuleList(
            (0): Conv2d(8, 64, kernel_size=(1, 1), stride=(1, 1))
            (1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
            (2): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1))
          )
          (2): ModuleList(
            (0): Conv2d(8, 64, kernel_size=(1, 1), stride=(1, 1))
            (1): Conv2d(64, 96, kernel_size=(1, 1), stride=(1, 1))
            (2): Conv2d(96, 128, kernel_size=(1, 1), stride=(1, 1))
          )
        )
        (bn_blocks): ModuleList(
          (0): ModuleList(
            (0): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_r

In [ ]:
pred = dataset.get_matching_data(13)
print(pred.keys())
print(pred['keypoints0'].shape, pred['keypoints1'].shape, pred['descriptors0'].shape, pred['descriptors1'].shape, pred['scores0'].shape, pred['scores1'].shape)

In [7]:
# 3. 디스크립터 추출
net.double().eval()

for i in range(dataset.divided_seq_num - 1):
    pred = dataset.get_divided_data(i)
    for p in pred:
        pred[p] = pred[p].to(device)
    data = net.module.infer_desc(pred)
    for d in data:
        data[d] = data[d].detach().cpu()
    dataset.push_descriptor(i, data['desc'])

In [11]:
# 4. 매칭
# pred = dataset.get_matching_data(12)
pred = dataset.get_matching_data(13)
# pred = dataset.get_matching_data(12)
for p in pred:
    if type(pred[p]) == torch.Tensor:
        pred[p] = pred[p].to(device)
print(pred['keypoints0'].shape, pred['keypoints1'].shape, pred['descriptors0'].shape, pred['descriptors1'].shape)

data = net.module.infer_mdgat(pred, [pred['keypoints0'].shape[1]//2, None, pred['keypoints0'].shape[1]//2, None, pred['keypoints0'].shape[1]//4, None, pred['keypoints0'].shape[1]//4, None], [pred['keypoints1'].shape[1]//2, None, pred['keypoints1'].shape[1]//2, None, pred['keypoints1'].shape[1]//4, None, pred['keypoints1'].shape[1]//4, None])
pred = {**pred, **data}
print("pred's keys: ", pred.keys())

torch.Size([1, 512, 3]) torch.Size([1, 512, 3]) torch.Size([1, 128, 512]) torch.Size([1, 128, 512])
pred's keys:  dict_keys(['keypoints0', 'keypoints1', 'descriptors0', 'descriptors1', 'cloud0', 'cloud1', 'gt_matches0', 'gt_matches1', 'rep', 'matches0', 'matches1', 'matching_scores0', 'matching_scores1'])


In [ ]:
kpts0, kpts1 = pred['keypoints0'][0].cpu().numpy(), pred['keypoints1'][0].cpu().numpy()
matches0, matches1, conf = pred['matches0'][0].cpu().detach().numpy(), pred['matches1'][0].cpu().detach().numpy(), pred['matching_scores0'][0].cpu().detach().numpy()
valid = matches0 > -1
mkpts0 = kpts0[valid]
mkpts1 = kpts1[matches0[valid]]

mutual0 = np.arange(len(matches0))[valid] == matches1[matches0[valid]]
mutual0 = np.arange(len(matches0))[valid][mutual0]
mutual1 = matches0[mutual0]
x = np.ones(len(matches1)) == 1
x[mutual1] = False
valid1 = matches1 > -1

mconf = conf[valid]

matches_gt, matches_gt1 = pred['gt_matches0'], pred['gt_matches1']
matches_gt[matches_gt == len(matches_gt1)] = -1
matches_gt1[matches_gt1 == len(matches_gt)] = -1
valid_gt = matches_gt > -1

print(matches0.shape, matches1.shape, conf.shape) # >> (3328,) (256,) (3328,)

print(matches0[669]) # matches0의 669번째 인덱스와의 매칭 결과 >> 1

print(matches1[:]) # matches1의 전체 결과 >> [  -1  669   -1  677 ...], 즉 matches1의 1번째 포인트가 matches0의 669번째 포인트와 매칭됨

print(mutual0) # >> [   9   10   13   15 ...]

print(mutual1) # >> [248 245  20 255 106 ...]

print(matches0[9], matches1[248]) # >> 248, 9, 즉 mutual0[i]와 mutual1[i]의 idx끼리 매칭 됨

In [ ]:
# 시각화
pcd_kp0 = o3d.geometry.PointCloud()
pcd_kp0.points = o3d.utility.Vector3dVector(kpts0)
pcd_kp0.paint_uniform_color([1, 0, 0])
pcd_kp1 = o3d.geometry.PointCloud()
pcd_kp1.points = o3d.utility.Vector3dVector(kpts1)
pcd_kp1.paint_uniform_color([0, 1, 0])

points = np.concatenate((np.array(pcd_kp0.points),np.array(pcd_kp1.points)), axis=0) # >> pcd_kp0에 pcd_kp1를 이어 붙힘
lines = []
for idx, match in enumerate(mutual0): # mutual0의 값
    # lines.append([match, match + 1])
    # lines.append([mutual1[idx] + len(kpts0), mutual1[idx] + len(kpts0)])
    lines.append([match, mutual1[idx] + len(kpts0)])
colors = [[0, 1, 0] for _ in range(len(lines))] # lines are shown in green
# print(points[lines[0][0]], points[lines[0][1]])
# print(pcd_kp0.points[lines[0][0]], pcd_kp1.points[lines[0][1] - len(pcd_kp0.points)])
line_set = o3d.geometry.LineSet(
    points=o3d.utility.Vector3dVector((points)),
    lines=o3d.utility.Vector2iVector(lines),
)
line_set.colors = o3d.utility.Vector3dVector(colors)
# o3d.visualization.draw_geometries([pcd0,pcd1,line_set])

o3d.visualization.draw_geometries([pcd_kp0, pcd_kp1, line_set])

In [ ]:
ttt = 2.0

# 시각화
kpts0, kpts1 = pred['keypoints0'][0].cpu().numpy(), pred['keypoints1'][0].cpu().numpy()
pcd_kp0 = o3d.geometry.PointCloud()
pcd_kp0.points = o3d.utility.Vector3dVector(kpts0)
pcd_kp0.paint_uniform_color([1, 0, 0])
pcd_kp1 = o3d.geometry.PointCloud()
pcd_kp1.points = o3d.utility.Vector3dVector(kpts1)
pcd_kp1.paint_uniform_color([0, 1, 0])

desc0 = pred['descriptors0'][0].cpu().detach().numpy().T
desc1 = pred['descriptors1'][0].cpu().detach().numpy().T
dists = cdist(desc0, desc1)
min0 = np.argmin(dists, axis=0)
min1 = np.argmin(dists, axis=1)
min0v = np.min(dists, axis=1)
min0f = min1[min0v < ttt]
match0, match1 = -1 * np.ones((len(desc0)), dtype=np.int16), -1 * np.ones((len(desc1)), dtype=np.int16)
match0[min0v < ttt] = min0f
min1v = np.min(dists, axis=0)
min1f = min0[min1v < ttt]
match1[min1v < ttt] = min1f
mutual0 = np.arange(len(matches0))[valid] == matches1[matches0[valid]]
mutual0 = np.arange(len(matches0))[valid][mutual0]
mutual1 = matches0[mutual0]

# 시각화
pcd_kp0 = o3d.geometry.PointCloud()
pcd_kp0.points = o3d.utility.Vector3dVector(kpts0)
pcd_kp0.paint_uniform_color([1, 0, 0])
pcd_kp1 = o3d.geometry.PointCloud()
pcd_kp1.points = o3d.utility.Vector3dVector(kpts1)
pcd_kp1.paint_uniform_color([0, 1, 0])

points = np.concatenate((np.array(pcd_kp0.points),np.array(pcd_kp1.points)), axis=0) # >> pcd_kp0에 pcd_kp1를 이어 붙힘
lines = []
for idx, match in enumerate(mutual0): # mutual0의 값
    lines.append([match, mutual1[idx] + len(kpts0)])
colors = [[0, 1, 0] for _ in range(len(lines))] # lines are shown in green
# print(points[lines[0][0]], points[lines[0][1]])
# print(pcd_kp0.points[lines[0][0]], pcd_kp1.points[lines[0][1] - len(pcd_kp0.points)])
line_set = o3d.geometry.LineSet(
    points=o3d.utility.Vector3dVector((points)),
    lines=o3d.utility.Vector2iVector(lines),
)
line_set.colors = o3d.utility.Vector3dVector(colors)
# o3d.visualization.draw_geometries([pcd0,pcd1,line_set])

o3d.visualization.draw_geometries([pcd_kp0, pcd_kp1, line_set])


In [ ]:
print(mutual0)
print(mutual1)